In [1]:
from pydriller import Repository

## Experiments

In [7]:
"""
for commit in Repository('./seatunnel/').traverse_commits():
    print(commit.hash)
    print(commit.msg)
    print(commit.author.name)

    for file in commit.modified_files:
        print(file.filename, ' has changed')
"""

"\nfor commit in Repository('./seatunnel/').traverse_commits():\n    print(commit.hash)\n    print(commit.msg)\n    print(commit.author.name)\n\n    for file in commit.modified_files:\n        print(file.filename, ' has changed')\n"

In [8]:
# # of files end with .java
# # of those files which contain Test

"""
seatunnel:
total commits = 3692
total commits which modify a java file = 1828
"""

'\nseatunnel:\ntotal commits = 3692\ntotal commits which modify a java file = 1828\n'

In [9]:
# visited_files = {} # filename: commit_index

In [10]:
# commits = Repository('./seatunnel/', only_in_branch='dev', only_modifications_with_file_types=['.java']).traverse_commits()

In [11]:
# for index, commit in enumerate(commits):
#     print(index)
#     for modified_file in commit.modified_files:
#         modified_filename = modified_file.filename
#         if modified_file.change_type.name =='ADD' and modified_filename.endswith('.java'):
#             visited_files[modified_filename] = index


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [12]:
# visited_files

{'ConfigVisitorImpl.java': 0,
 'Test.java': 0,
 'PluginDoc.java': 7,
 'PluginDocCommand.java': 7,
 'PluginDocMarkdownRender.java': 7,
 'PluginDocRuntimeException.java': 12,
 'ConfigRuntimeException.java': 42,
 'StringTemplate.java': 67,
 'CompressionUtils.java': 373,
 'AsciiArt.java': 57,
 'ExposeSparkConf.java': 55,
 'OpentsdbCallBack.java': 27,
 'HttpAsyncClient.java': 27,
 'HttpClientFactory.java': 27,
 'HttpClientService.java': 27,
 'TypesafeConfigUtils.java': 30,
 'SparkSturctTypeUtil.java': 126,
 'JdbcConnectionPoll.java': 34,
 'WaterdropDoctor.java': 36,
 'ClassLoaderUtils.java': 36,
 'WaterdropRuntimeException.java': 37,
 'UserRuntimeException.java': 38,
 'BaseSink.java': 39,
 'BaseSource.java': 39,
 'BaseTransform.java': 39,
 'RuntimeEnv.java': 39,
 'CheckResult.java': 39,
 'Plugin.java': 39,
 'Application.java': 39,
 'ConfigParser.java': 317,
 'FlinkBatchEnv.java': 39,
 'AbstractFlinkBatchSink.java': 39,
 'AbstractFlinkBatchSource.java': 39,
 'AbstractFlinkBatchTransform.java

In [19]:
# test_files = 0
# source_files = 0

# for filename in visited_files.keys():
#     if 'Test' in filename:
#         test_files += 1
#     else:
#         source_files += 1

In [20]:
# test_files

260

In [21]:
# source_files

2705

In [23]:
# tdd = 0
# same = 0
# non_tdd = 0

# for filename in visited_files.keys():
#     if 'Test' in filename:
#         source_filename = filename.replace('Test', '')
#         if source_filename in visited_files:
#             if visited_files[filename]<visited_files[source_filename]:
#                 tdd += 1
#             elif visited_files[filename]==visited_files[source_filename]:
#                 same += 1
#             else:
#                 non_tdd += 1

In [24]:
# tdd

10

In [25]:
# same

65

In [26]:
# non_tdd

66

## Main

In [36]:
def mine(commits, verbose=False):
    java_files_with_commit_index = {}
    merge_commit_indexes = {}
    total = 0
    for index, commit in enumerate(commits):
        if verbose and index%10==0:
            print(index)
        if commit.merge:
            merge_commit_indexes.add(index)
        for modified_file in commit.modified_files:
            if modified_file.change_type.name =='ADD' and modified_file.filename.endswith('.java'):
                java_files_with_commit_index[modified_file.filename] = index
        total = index
    return java_files_with_commit_index, merge_commit_indexes, total

In [28]:
def number_of_test_files(java_files_with_commit_index):
    test_files = 0
    for filename in java_files_with_commit_index.keys():
        if 'Test' in filename:
            test_files += 1
    return test_files


def amount_of_tdd(java_files_with_commit_index):
    tdd, same, non_tdd, error = 0, 0, 0, 0
    for filename in java_files_with_commit_index.keys():
        if 'Test' in filename:
            source_filename = filename.replace('Test', '')
            if source_filename in java_files_with_commit_index:
                if java_files_with_commit_index[filename]<java_files_with_commit_index[source_filename]:
                    tdd += 1
                elif java_files_with_commit_index[filename]==java_files_with_commit_index[source_filename]:
                    same += 1
                else:
                    non_tdd += 1
            else:
                error += 1
    return tdd, same, non_tdd, error

def analyse(java_files_with_commit_index):
    test_files = number_of_test_files(java_files_with_commit_index)
    tdd, same, non_tdd, error = amount_of_tdd(java_files_with_commit_index)
    return test_files, tdd, same, non_tdd, error

In [30]:
def print_analysis(total_commits, number_of_merge_commits, number_of_files, test_files, tdd, same, non_tdd, error):
    print(f'Number Of Commits Where Java Files Were Modified: {total_commits}')
    print(f'Number Of Merge Commits: {number_of_merge_commits}')
    print(f'Number Of Java Files: {number_of_files}')
    print(f'Number Of Test Files: {test_files}')
    print(f'Number Of Source Files: {number_of_files-test_files}')
    print(f'Number Of Times Test File Created Before Source File (TDD): {tdd}')
    print(f'Number Of Times Test File Created In The Same Commit As The Source File: {same}')
    print(f'Number Of Times Test File Created After Source File (Non-TDD): {non_tdd}')
    print(f'Number Of Times Matching Source File For A Test File Was Not Found (Error): {error}')
    print('-'*200)

In [34]:
def run(urls_with_main_branch, verbose=False):
    for url in urls_with_main_branch.keys():
        print(url)
        commits = Repository(url, only_in_branch=urls_with_main_branch[url], only_modifications_with_file_types=['.java']).traverse_commits()
        java_files_with_commit_index, merge_commit_indexes, total_commits = mine(commits, verbose=verbose)
        test_files, tdd, same, non_tdd, error = analyse(java_files_with_commit_index)
        if verbose:
            print_analysis(total_commits, len(merge_commit_indexes), len(java_files_with_commit_index.keys()), test_files, tdd, same, non_tdd, error)

In [32]:
urls = {
    './seatunnel/' : 'dev',
    './dubbo/' : '3.2',
    './shenyu/' : 'master',
    './ozone/' : 'master'
}

In [35]:
run(urls, verbose=True)

./seatunnel/
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
2